<a href="https://colab.research.google.com/github/limshaocong/SysBERT/blob/main/Sequence_classification_PROMISE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preliminaries

In [1]:
! pip install datasets transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 346 kB 8.3 MB/s 
     |████████████████████████████████| 4.2 MB 62.2 MB/s 
     |████████████████████████████████| 140 kB 88.2 MB/s 
     |████████████████████████████████| 86 kB 6.9 MB/s 
     |████████████████████████████████| 86 kB 5.8 MB/s 
     |████████████████████████████████| 212 kB 64.6 MB/s 
     |████████████████████████████████| 1.1 MB 54.5 MB/s 
     |████████████████████████████████| 596 kB 86.7 MB/s 
     |████████████████████████████████| 127 kB 74.7 MB/s 
     |████████████████████████████████| 6.6 MB 56.6 MB/s 
     |████████████████████████████████| 271 kB 78.5 MB/s 
     |████████████████████████████████| 144 kB 97.3 MB/s 
     |████████████████████████████████| 94 kB 4.0 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.

In [2]:
! nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-231f8b1b-286d-8098-c2de-eb4668a8f0a0)


# Import & Pre-process Data

Clone the repository containing the labeled requirements data.

In [3]:
! git clone https://github.com/limshaocong/SysBERT/

Cloning into 'SysBERT'...
remote: Enumerating objects: 459, done.
remote: Counting objects: 100% (459/459), done.
remote: Compressing objects: 100% (438/438), done.
remote: Total 459 (delta 42), reused 396 (delta 11), pack-reused 0
Receiving objects: 100% (459/459), 3.64 MiB | 18.63 MiB/s, done.
Resolving deltas: 100% (42/42), done.


This sequence classification task is performed using the labeled PROMISE dataset. The targte variable is denoted by the 'is_functional' column; 1 = functional requirement, 0 = non-functional requirement. The train, validation and test datasets are created by stratified sampling in a 70/15/15 ratio. The data is imported as a HuggingFace [Dataset](https://huggingface.co/docs/datasets/access) object for ease of downstream manipulation.

In [4]:
from datasets import load_dataset

train_path = '/content/SysBERT/Requirements/Labeled/Sequence_Classification/Pre-processed/Full/train.csv'
val_path = '/content/SysBERT/Requirements/Labeled/Sequence_Classification/Pre-processed/Full/val.csv'
test_path = '/content/SysBERT/Requirements/Labeled/Sequence_Classification/Pre-processed/Full/test.csv'

promise = load_dataset(
    'csv',
    data_files = {
        'train': train_path,
        'val' : val_path,
        'test': test_path
        }
)

promise

Using custom data configuration default-afabe42ae3040185


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-afabe42ae3040185/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['reqs', 'is_functional'],
        num_rows: 669
    })
    val: Dataset({
        features: ['reqs', 'is_functional'],
        num_rows: 143
    })
    test: Dataset({
        features: ['reqs', 'is_functional'],
        num_rows: 144
    })
})

To transform natural language requirements into a BERT-compatible format, the text must first be tokenized. This is performed using a pre-trained tokenizer.

In [5]:
from transformers import AutoTokenizer

model_checkpoint = 'bert-base-cased'

tokenizer = AutoTokenizer.from_pretrained(
    model_checkpoint,
    use_fast = True
)

def encode(requirements):
    return tokenizer(requirements['reqs'], truncation = True, max_length = 128)

promise_tokenized = promise.map(encode, batched = True)

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [6]:
sample = promise['train'][0]

print(sample)
encode(sample)

{'reqs': 'The RFS system should be available 24/7 especially during the budgeting period. The RFS system shall be available 90% of the time all year and 98% during the budgeting period. 2% of the time the system will become available within 1 hour of the time that the situation is reported.', 'is_functional': 0}


{'input_ids': [101, 1109, 24695, 1708, 1449, 1431, 1129, 1907, 1572, 120, 128, 2108, 1219, 1103, 4788, 1158, 1669, 119, 1109, 24695, 1708, 1449, 4103, 1129, 1907, 3078, 110, 1104, 1103, 1159, 1155, 1214, 1105, 5103, 110, 1219, 1103, 4788, 1158, 1669, 119, 123, 110, 1104, 1103, 1159, 1103, 1449, 1209, 1561, 1907, 1439, 122, 2396, 1104, 1103, 1159, 1115, 1103, 2820, 1110, 2103, 119, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [7]:
promise_tokenized

DatasetDict({
    train: Dataset({
        features: ['reqs', 'is_functional', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 669
    })
    val: Dataset({
        features: ['reqs', 'is_functional', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 143
    })
    test: Dataset({
        features: ['reqs', 'is_functional', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 144
    })
})

In [8]:
from transformers import DataCollatorWithPadding

batch_size = 16
output_col = 'is_functional'

data_collator = DataCollatorWithPadding(
    tokenizer = tokenizer,
    return_tensors = 'tf'
)

batched_train_ds = promise_tokenized['train'].to_tf_dataset(
    columns = ['attention_mask', 'input_ids', 'token_type_ids'],
    label_cols = [output_col],
    shuffle = False,
    drop_remainder = False,
    collate_fn = data_collator,
    batch_size = batch_size
)

batched_val_ds = promise_tokenized['val'].to_tf_dataset(
    columns = ['attention_mask', 'input_ids', 'token_type_ids'],
    label_cols = [output_col],
    shuffle = False,
    drop_remainder = False,
    collate_fn = data_collator,
    batch_size = batch_size
)

batched_test_ds = promise_tokenized['test'].to_tf_dataset(
    columns = ['attention_mask', 'input_ids', 'token_type_ids'],
    label_cols = [output_col],
    shuffle = False,
    drop_remainder = False,
    collate_fn = data_collator,
    batch_size = batch_size
)

# Model Fine-tuning

In [16]:
import tensorflow as tf
from transformers import TFAutoModelForSequenceClassification

tf.keras.utils.set_random_seed(1234)

model = TFAutoModelForSequenceClassification.from_pretrained(
    model_checkpoint,
    num_labels = 2
)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
for layer in model.layers[:]:
    print(layer, layer.trainable)

<transformers.models.bert.modeling_tf_bert.TFBertMainLayer object at 0x7f9e201adf50> True
<keras.layers.core.dropout.Dropout object at 0x7f9db86cb410> True
<keras.layers.core.dense.Dense object at 0x7f9db86cbf10> True


In [17]:
from transformers import models

frozen_layers = range(0, 8)

for layer in model.layers[:]:
  
  # Replace transformers.models.bert.modeling_tf_bert.TFBertMainLayer
  # with the corresponding MainLayer name from the previous code output
  if isinstance(layer, models.bert.modeling_tf_bert.TFBertMainLayer):
    
    for idx, layer in enumerate(layer.encoder.layer):
      
      if idx in frozen_layers:
        layer.trainable = False
      
      # Confirm the chosen layers are frozen
      print(layer, layer.trainable)

<transformers.models.bert.modeling_tf_bert.TFBertLayer object at 0x7f9d53e54f10> False
<transformers.models.bert.modeling_tf_bert.TFBertLayer object at 0x7f9d921b4e50> False
<transformers.models.bert.modeling_tf_bert.TFBertLayer object at 0x7f9d57fd17d0> False
<transformers.models.bert.modeling_tf_bert.TFBertLayer object at 0x7f9d57fe6190> False
<transformers.models.bert.modeling_tf_bert.TFBertLayer object at 0x7f9d57ff4d10> False
<transformers.models.bert.modeling_tf_bert.TFBertLayer object at 0x7f9d597cecd0> False
<transformers.models.bert.modeling_tf_bert.TFBertLayer object at 0x7f9d597e5b90> False
<transformers.models.bert.modeling_tf_bert.TFBertLayer object at 0x7f9d597fcad0> False
<transformers.models.bert.modeling_tf_bert.TFBertLayer object at 0x7f9d59793a10> True
<transformers.models.bert.modeling_tf_bert.TFBertLayer object at 0x7f9d597cec10> True
<transformers.models.bert.modeling_tf_bert.TFBertLayer object at 0x7f9d59743950> True
<transformers.models.bert.modeling_tf_bert.TFB

In [18]:
model.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  108310272 
                                                                 
 dropout_75 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 108,311,810
Trainable params: 51,608,834
Non-trainable params: 56,702,976
_________________________________________________________________


In [19]:
from transformers import create_optimizer

num_epochs = 5

batches_per_epoch = len(promise_tokenized['train']) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)

optimizer, schedule = create_optimizer(
    init_lr = 2e-5,
    num_warmup_steps = 0,
    num_train_steps = total_train_steps
)

loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True)

model.compile(
    optimizer = optimizer,
    loss = loss,
    metrics = tf.metrics.SparseCategoricalAccuracy()
)

In [20]:
model.fit(
    batched_train_ds,
    validation_data = batched_val_ds,
    epochs = num_epochs
)

Epoch 1/5
42/42 [==============================] - 21s 187ms/step - loss: 0.6122 - sparse_categorical_accuracy: 0.6771 - val_loss: 0.4562 - val_sparse_categorical_accuracy: 0.8182
Epoch 2/5
42/42 [==============================] - 5s 123ms/step - loss: 0.3591 - sparse_categorical_accuracy: 0.8759 - val_loss: 0.3828 - val_sparse_categorical_accuracy: 0.8322
Epoch 3/5
42/42 [==============================] - 5s 123ms/step - loss: 0.2380 - sparse_categorical_accuracy: 0.9178 - val_loss: 0.3485 - val_sparse_categorical_accuracy: 0.8601
Epoch 4/5
42/42 [==============================] - 5s 123ms/step - loss: 0.1973 - sparse_categorical_accuracy: 0.9372 - val_loss: 0.3572 - val_sparse_categorical_accuracy: 0.8601
Epoch 5/5
42/42 [==============================] - 5s 123ms/step - loss: 0.1647 - sparse_categorical_accuracy: 0.9507 - val_loss: 0.3454 - val_sparse_categorical_accuracy: 0.8531
